# Analyzing Bonus Wins and Losses Feature

## Introuction

### What is the bonus win/loss feature?

There is a new feature in ESPN called 'Bonus Wins and Losses'. With this feature on, teams will be awarded an additional Win or Loss based on their matchup score versus the rest of the league. If a team's mathup score is in the top half of the league for the week, that team gets an additional Win. If a team's matchup score is in the bottom half of the league for the week, that team gets an additional Loss.

Each week of the regular season, your team will go 2-0, 0-2 or 1-1. 

### What is the puropose of this?

According to ESPN, this feature is supposed to help reward those teams that may score a ton of points, but manage to face the high-scoring teams more often than not. This feature helps the best performing teams teams reach the playoffs, and lowers the chance that good teams are eliminated based on their opponent's performances rather than their own. Instead of H2H being the only thing that matters, your weekly score vs. your other league opponents also matters. This lessens the randomness of fantasy football and helps the likelihood that the best teams will make it into the playoffs. 

## Hypothesis

### How would this have affected our league last year?

My hypothesis is that this wouldn't affect our league too much. If anything, it might just shift a couple of teams around in the standings. The following analysis will determine the results. 

## Analysis

### Imports and reading in team data

In [1]:
# import libraries
import pandas as pd
import numpy as np

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# import 2021 team weekly scores
df = pd.read_excel('team_scoring_data_2021.xlsx')

In [3]:
df

,Team,Owner,Week,Score,H2H,Season
0,Uncle Rozay,Chris Rosales,1,139.25,L,2021
1,Uncle Rozay,Chris Rosales,2,107.70,W,2021
2,Uncle Rozay,Chris Rosales,3,153.05,W,2021
3,Uncle Rozay,Chris Rosales,4,99.60,W,2021
4,Uncle Rozay,Chris Rosales,5,144.55,L,2021
...,...,...,...,...,...,...
163,Team Eddie,Eddie Jaramillo,10,114.95,L,2021
164,Team Eddie,Eddie Jaramillo,11,116.50,L,2021
165,Team Eddie,Eddie Jaramillo,12,149.40,W,2021
166,Team Eddie,Eddie Jaramillo,13,128.80,W,2021


### Data Validation

Ensure no typos and all data matches with league history.

confirm win/loss records

confirm points for scored

In [4]:
# get a list of unique team names
teams = list(df['Team'].unique())

In [5]:
teams

['Uncle Rozay',
 "Fauci's Quaranteam",
 '2 Sheets 2 Da WIN',
 'Resistance Is Futile',
 'Beevah The Diva',
 'Really need to PiSsSsSsSsSs$',
 'Take Mahomes, Country Road',
 'Papa Eye clk, clk, clk',
 'Hooked on a Theilen',
 "Jerry's Glory Hole",
 "C'mon mannnnn J.king",
 'Team Eddie']

In [6]:
# create blank df
standings = pd.DataFrame(columns={'team','wins','losses', 'win_percent','points_for'})

# populate df with each team's info
for team in teams:
    wins = df[(df['Team'] == team) & (df['H2H'] == 'W') ]['H2H'].count()
    losses = df[(df['Team'] == team) & (df['H2H'] == 'L') ]['H2H'].count()
    points_for = df[df['Team'] == team]['Score'].sum()
    win_percent = round(wins/(wins+losses) * 100)
    team_name = team
    
    # put info into dictionary
    team_dict = {'wins': [wins],
        'team':[team_name],
        'losses':[losses],
        'win_percent' : [win_percent],
        'points_for' : [points_for]
       }
    # convert dictionary to df and append to standings df
    df_2 = pd.DataFrame(team_dict)
    standings = standings.append(df_2, ignore_index = True, sort = True)
    
    

In [7]:
# rearrange columns
cols = ['team','wins','losses','win_percent','points_for']
standings = standings[cols]
standings.sort_values(by=['wins','points_for'], ascending = False)

,team,wins,losses,win_percent,points_for
0,Uncle Rozay,12,2,86,1878.85
1,Fauci's Quaranteam,10,4,71,1908.50
7,"Papa Eye clk, clk, clk",9,5,64,1845.85
11,Team Eddie,8,6,57,1711.60
5,Really need to PiSsSsSsSsSs$,7,7,50,1825.00
4,Beevah The Diva,7,7,50,1763.60
10,C'mon mannnnn J.king,7,7,50,1737.75
2,2 Sheets 2 Da WIN,7,7,50,1575.00
3,Resistance Is Futile,6,8,43,1611.20
6,"Take Mahomes, Country Road",5,9,36,1713.20


## Adding an extra win/loss for each week



### Would this have made a difference in our league last year?

#### Logic to create additional wins

Test logic for adding wins and losses: 
- Get the team's score for a given week. 
- Get top 6 scores for that week. 
- If team score is included in list of top 6 scores, that's an extra win. Otherwise, it's an extra loss.

In [8]:
# weekly scores for week 1
weekly_scores = list(df[df['Week'] == 1]['Score'].sort_values(ascending = False))

In [9]:
weekly_scores

[158.35,
 155.45,
 141.95,
 139.25,
 138.0,
 137.85,
 136.45,
 135.85,
 133.55,
 117.5,
 115.45,
 77.85]

In [10]:
# team score that week for sample team
weekly_team_score = df[(df['Team'] == 'Hooked on a Theilen') & (df['Week'] == 1)]['Score'].values[0]

In [11]:
weekly_team_score

136.45

### Function to add wins and losses to each team for each week

In [12]:
# create blank df
extra_games = pd.DataFrame(columns={'team','extra_wins','extra_losses'})

# populate df with each team's info
for team in teams:
    extra_wins = 0
    extra_losses = 0
    team_name = team
    for week in list(df['Week'].unique()):
        # weekly scores
        weekly_scores = list(df[df['Week'] == week]['Score'].sort_values(ascending = False))
        # team score that week
        weekly_team_score = df[(df['Team'] == team) & (df['Week'] == week)]['Score'].values[0]
        if weekly_team_score in weekly_scores[:6]:
            extra_wins += 1
        else: 
            extra_losses += 1
    
    # put info into dictionary
    team_dict = {'extra_wins': [extra_wins],
        'team':[team_name],
        'extra_losses':[extra_losses]
       }
    # convert dictionary to df and append to standings df
    df_3 = pd.DataFrame(team_dict)
    extra_games = extra_games.append(df_3, ignore_index = True, sort = True)
    
    

In [13]:
extra_games

,extra_losses,extra_wins,team
0,3,11,Uncle Rozay
1,4,10,Fauci's Quaranteam
2,8,6,2 Sheets 2 Da WIN
3,8,6,Resistance Is Futile
4,8,6,Beevah The Diva
5,4,10,Really need to PiSsSsSsSsSs$
6,8,6,"Take Mahomes, Country Road"
7,4,10,"Papa Eye clk, clk, clk"
8,13,1,Hooked on a Theilen
9,10,4,Jerry's Glory Hole


In [14]:
final = pd.merge(standings, extra_games, on = 'team')

### New standings

In [15]:
final['total_wins'] = final['wins'] + final['extra_wins']
final['total_losses'] = final['losses'] + final['extra_losses']
#final = ['team','wins','losses','win_percent','points_for']
final.sort_values(by=['total_wins','points_for'], ascending = False)

,team,wins,losses,win_percent,points_for,extra_losses,extra_wins,total_wins,total_losses
0,Uncle Rozay,12,2,86,1878.85,3,11,23,5
1,Fauci's Quaranteam,10,4,71,1908.50,4,10,20,8
7,"Papa Eye clk, clk, clk",9,5,64,1845.85,4,10,19,9
5,Really need to PiSsSsSsSsSs$,7,7,50,1825.00,4,10,17,11
10,C'mon mannnnn J.king,7,7,50,1737.75,6,8,15,13
11,Team Eddie,8,6,57,1711.60,8,6,14,14
4,Beevah The Diva,7,7,50,1763.60,8,6,13,15
2,2 Sheets 2 Da WIN,7,7,50,1575.00,8,6,13,15
3,Resistance Is Futile,6,8,43,1611.20,8,6,12,16
6,"Take Mahomes, Country Road",5,9,36,1713.20,8,6,11,17


### Old Standings

In [16]:
standings.sort_values(by=['wins','points_for'], ascending = False)

,team,wins,losses,win_percent,points_for
0,Uncle Rozay,12,2,86,1878.85
1,Fauci's Quaranteam,10,4,71,1908.50
7,"Papa Eye clk, clk, clk",9,5,64,1845.85
11,Team Eddie,8,6,57,1711.60
5,Really need to PiSsSsSsSsSs$,7,7,50,1825.00
4,Beevah The Diva,7,7,50,1763.60
10,C'mon mannnnn J.king,7,7,50,1737.75
2,2 Sheets 2 Da WIN,7,7,50,1575.00
3,Resistance Is Futile,6,8,43,1611.20
6,"Take Mahomes, Country Road",5,9,36,1713.20


# Conclusion:

- This feature helped the top 3 teams solidify their lead, and pushed the bottom 3 teams down even lower.
- The six middle pack teams, which were only two games apart, have more clarity and separation. This separation, while not exactly in line with total points scored, correlates to put the higher scoring teams higher than the lower scoring teams.
- This new feature does NOT guarantee that only points_for matters. Matchups still matter a lot! Just not 100%.

#### Would I recommend? 
Yes I would. I think this makes fantasy football more fair and helps teams make the playoffs based on their own team's performance rather than their opponents. I believe this feature also makes fantasy football more fun. The ability to get two wins in one week will potentially make the season standings more volatile, allowing a team that started poorly a greater chance to catch up and make the playoffs. It will also increase engagement, as the only players that matter are not only in your matchup, but across other teams as well. Example: MNF rolls around and you already lost your matchup to the highest scoring team that week. Perhaps two other teams have players going tonight that could affect whether or not you get an extra W or L. 
